In [2]:
import pandas as pd
from data import load_debt_data, total_annual_debt, total_annual_unemployment, filter_by_year, filter_by_years, filter_by_states

In [3]:
df = pd.read_csv('data/debt_92-05.csv', sep=';')
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df['time'] = pd.to_datetime(df['time']).dt.year

In [4]:
# get total annual debt by year for Berlin on a state level
filtered_df = df[df['1_variable_attribute_label'] == "Berlin"]
filtered_df = filtered_df[filtered_df['2_variable_attribute_label'] == "Länder"]
filtered_df['total_annual_debt'] = filtered_df.groupby('time')['value'].transform('sum')
filtered_df = filtered_df.groupby('time')['total_annual_debt'].agg(lambda x: list(x)[0]).reset_index()
filtered_df

,time,total_annual_debt
0,1992,13069.0
1,1993,16053.0
2,1994,18454.0
3,1995,23700.0
4,1996,26911.0
5,1997,29000.0
6,1998,31211.0
7,1999,33231.0
8,2000,34936.0
9,2001,39778.0


In [1]:
from data import load_population_density, population_from_density

df= load_population_density()

pop= population_from_density()

print(pop[pop['year']=='2015-12-31'])

          year  Baden-Württemberg    Bayern   Berlin  Brandenburg  Bremen  \
20  2015-12-31           10868608  12839918  3521616      2476068  645996   

    Hamburg   Hessen  Mecklenburg-Vorpommern  Niedersachsen  \
20  1786330  6186695                 1598937        7904588   

    Nordrhein-Westfalen  Rheinland-Pfalz  Saarland  Sachsen  Sachsen-Anhalt  \
20             17859492          4048788    996772  4069273         2248950   

    Schleswig-Holstein  Thüringen  
20             2853103    2167048  


In [1]:
from data import normalized_debt_per_capita, normalized_unemployment_per_capita


debt_norm = normalized_debt_per_capita()
unemp_norm = normalized_unemployment_per_capita()

print(debt_norm.head())
print(unemp_norm.head())


               state  year  debt_per_person_eur
0  Baden-Württemberg  1995              2481.92
1  Baden-Württemberg  1996              2632.99
2  Baden-Württemberg  1997              2727.84
3  Baden-Württemberg  1998              2831.82
4  Baden-Württemberg  1999              2893.37
               state  year  unemployment_rate_percent
0  Baden-Württemberg  1995                       3.18
1  Baden-Württemberg  1996                       3.41
2  Baden-Württemberg  1997                       3.67
3  Baden-Württemberg  1998                       3.37
4  Baden-Württemberg  1999                        3.1
